In [2]:
import cv2 as cv
import numpy as np

# Facial detection using OpenCV and Haar Cascades

In [3]:

# test function
def face_detect(img):
    # convert to gray
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # load classifier
    face_detector = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    # draw rectangle
    for (x, y, w, h) in faces:
        cv.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1)
    # show image
    cv.imshow('result', img)


# read image
# img = cv.imread('/Users/huxinyu/Desktop/UROP2023/images.jpg')
# face_detect()

# use webcam
cap = cv.VideoCapture(0)




while True:
    flag, frame = cap.read()

    # 如果已经播放完毕，则退出
    if not flag:
        break
    face_detect(frame)




    if cv.waitKey(1) & 0xFF == ord('q'):
        break

#cap.release()
cv.destroyAllWindows()

# Record Faces

In [45]:
cap = cv.VideoCapture(0)

flag, num = 1,31


while (cap.isOpened()):
    suc, frame = cap.read()
    cv.imshow("capture", frame)
    
    # control the frequency of saving images
    k = cv.waitKey(1) & 0xFF
    if k == ord('s'):
        cv.imwrite("/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_image/" + str(num) + ".julie" + ".jpg", frame)
        num += 1
        print("save image successfully!")
    elif k == ord('q'):
        break
cap.release()
cv.destroyAllWindows()


save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!
save image successfully!


# Train the Recoded Faces

In [4]:
import os
from PIL import Image
import numpy as np


def get_images_and_labels(path):

    # data for the faces
    images = []
    # data for the labels
    labels = []
    # get the path of all the files in the folder
    # image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    import glob
    image_paths = [f for f in glob.glob('/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_image/*.jpg')]
    # if '/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_image/' + '.DS_Store' in image_paths:
    #     image_paths.remove('/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_image/' + '.DS_Store')
    face_detect = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # go through all the image paths and load the images
    for image_path in image_paths:
        # if image_path == '/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_image/' + '.DS_Store':
        #     continue
        # load the image and convert it to grayscale, PIL has nine different modes
        # L: 8-bit pixels, black and white
        image = Image.open(image_path).convert('L')
        # convert PIL image to numpy array
        image_numpy = np.array(image, 'uint8')
        # get the data of the image
        faces = face_detect.detectMultiScale(image_numpy, scaleFactor=1.1, minNeighbors=5)
        # get the label of the image
        label = int(os.path.split(image_path)[1].split(".")[0].replace(" ", "-"))
        # append the image to the list of images
        for (x, y, w, h) in faces:
            images.append(image_numpy[y:y+h, x:x+w])
            labels.append(label)
            cv.imshow('result', image_numpy[y:y+h, x:x+w])
            cv.waitKey(100)
    print(labels)
    # return the images list and labels list
    return images, labels



path = '/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_image/'
images, labels = get_images_and_labels(path)
recognizer = cv.face.LBPHFaceRecognizer_create()
recognizer.train(images, np.array(labels))
# os.mkdir('training_data')
recognizer.write('training_data/training_data.yml')
    
    

[26, 32, 21, 17, 4, 16, 5, 34, 20, 39, 33, 27, 27, 7, 7, 14, 14, 42, 6, 15, 3, 10, 24, 24, 30, 29, 22, 2, 41, 11, 9, 31, 25, 25, 13, 28, 28, 12, 23, 1, 40, 8]


In [ ]:
import numpy as np
import os
import urllib
import urllib.request

# load the recognizer file
recognizer = cv.face.LBPHFaceRecognizer_create()
recognizer.read('/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_data/training_data.yml')

names = []
warning_times = 0

# some warning functions for recognizing unknown faces
# def warning():

def face_detect_demo(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    face_detector = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    face = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for x, y, w, h in face:
        cv.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1)
        cv.circle(img, (x+w//2, y+h//2), min(w//2, h//2), (0, 0, 255), 1)

        ids, confidence = recognizer.predict(gray[y:y+h, x:x+w])
        if confidence < 100:
            cv.putText(img, names[ids-1], (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv.LINE_AA)
        else:
            cv.putText(img, 'unknown', (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv.LINE_AA)
        
    cv.imshow('result', img)

def name():
    path = "/Users/huxinyu/Desktop/UROP2023/Facial Detecting/training_image/"
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    for image_path in image_paths:
        name = str(os.path.split(image_path)[1].split(".",2)[1])
        names.append(name)

cap = cv.VideoCapture(0)
name()
while True:
    ret, frame = cap.read()
    if not ret:
        break
    face_detect_demo(frame)
    if ord('q') == cv.waitKey(10):
        break

cap.release()
cv.destroyAllWindows()

In [8]:
import cv2
import imutils
import numpy as np

def detect(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    face = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    person = 1
    for x,y,w,h in face:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 1)
        cv2.putText(frame, f'person {person}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
        person += 1
    
    cv2.putText(frame, 'Status : Detecting ', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.putText(frame, f'Total Persons : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.imshow('output', frame)

    # return frame

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    detect(frame)
    if ord('q') == cv2.waitKey(10):
        break

cap.release()
cv2.destroyAllWindows()



# def detectByCamera(writer):   
#     video = cv2.VideoCapture(0)
#     print('Detecting people...')

#     while True:
#         check, frame = video.read()

#         frame = detect(frame)
#         if writer is not None:
#             writer.write(frame)

#         key = cv2.waitKey(1)
#         if key == ord('q'):
#                 break

#     video.release()
#     cv2.destroyAllWindows()


# def humanDetector(args):
  
#     camera = True 
#     writer = cv2.VideoWriter('output',cv2.VideoWriter_fourcc(*'MJPG'), 10, (600,600))

#     if camera:
#         print('[INFO] Opening Web Cam.')
#         detectByCamera(writer)


#     return args

# if __name__ == "__main__":
#     HOGCV = cv2.HOGDescriptor()
#     HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

#     humanDetector("true")

